In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

# Path to the TrainIJCNN2013 folder
train_folder_path = '/content/drive/MyDrive/DLP_A3/TrainIJCNN2013/TrainIJCNN2013'

# List all files and folders, but filter out only the folders
subfolders = [f for f in os.listdir(train_folder_path) if os.path.isdir(os.path.join(train_folder_path, f))]

# Print the list of subfolders
#print(subfolders)
print(len(subfolders))



43


In [3]:
categories = {
    '00': 'speed limit 20 (prohibitory)', '01': 'speed limit 30 (prohibitory)', '02': 'speed limit 50 (prohibitory)',
    '03': 'speed limit 60 (prohibitory)', '04': 'speed limit 70 (prohibitory)', '05': 'speed limit 80 (prohibitory)',
    '06': 'restriction ends 80 (other)', '07': 'speed limit 100 (prohibitory)', '08': 'speed limit 120 (prohibitory)',
    '09': 'no overtaking (prohibitory)', '10': 'no overtaking (trucks) (prohibitory)', '11': 'priority at next intersection (danger)',
    '12': 'priority road (other)', '13': 'give way (other)', '14': 'stop (other)', '15': 'no traffic both ways (prohibitory)',
    '16': 'no trucks (prohibitory)', '17': 'no entry (other)', '18': 'danger (danger)', '19': 'bend left (danger)',
    '20': 'bend right (danger)', '21': 'bend (danger)', '22': 'uneven road (danger)', '23': 'slippery road (danger)',
    '24': 'road narrows (danger)', '25': 'construction (danger)', '26': 'traffic signal (danger)', '27': 'pedestrian crossing (danger)',
    '28': 'school crossing (danger)', '29': 'cycles crossing (danger)', '30': 'snow (danger)', '31': 'animals (danger)',
    '32': 'restriction ends (other)', '33': 'go right (mandatory)', '34': 'go left (mandatory)', '35': 'go straight (mandatory)',
    '36': 'go right or straight (mandatory)', '37': 'go left or straight (mandatory)', '38': 'keep right (mandatory)',
    '39': 'keep left (mandatory)', '40': 'roundabout (mandatory)', '41': 'restriction ends (overtaking) (other)',
    '42': 'restriction ends (overtaking (trucks)) (other)'
}

In [4]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split

# Preprocessing function
def preprocess_image(img, target_size=(32, 32)):
    img = cv2.resize(img, target_size)  # Resize the image
    img = img / 255.0  # Normalize the image
    return img

# Load the images and labels
def load_data(base_path, categories):
    images = []
    labels = []

    for category_index, category_name in categories.items():
        category_folder = os.path.join(base_path, category_index)  # Use string folder names like '00', '01'
        image_files = [f for f in os.listdir(category_folder) if f.endswith('.ppm')]

        for image_file in image_files:
            img_path = os.path.join(category_folder, image_file)  # Full path to image
            img = cv2.imread(img_path)  # Read image
            img = preprocess_image(img)  # Preprocess image
            images.append(img)
            labels.append(int(category_index))  # Convert string index to int for label

    images = np.array(images)
    labels = np.array(labels)

    return images, labels

# Load the data
X, y = load_data(train_folder_path, categories)

# Split data into train and validation sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
!pip install ultralytics opencv-python-headless tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.0/887.0 kB 35.1 MB/s eta 0:00:00


# YOLO Model Code

In [13]:
import os
import cv2
import numpy as np
from glob import glob
import shutil
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from ultralytics import YOLO
import torch
from pathlib import Path


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [14]:
!pip install ultralytics opencv-python-headless tqdm

In [16]:


def create_dirs(base_path):
    """Create necessary directories for YOLO training."""
    dirs = ['images/train', 'images/val', 'labels/train', 'labels/val']
    for dir_path in dirs:
        os.makedirs(os.path.join(base_path, dir_path), exist_ok=True)

def convert_to_yolo_format(size, bbox):
    """
    Convert bounding box format from (x1, y1, x2, y2) to YOLO format (x_center, y_center, width, height).
    All values are normalized between 0 and 1.
    """
    dw = 1./size[0]
    dh = 1./size[1]

    # Original format: x1,y1,x2,y2
    x1, y1, x2, y2 = bbox

    # Calculate YOLO format
    w = x2 - x1
    h = y2 - y1
    x_center = x1 + w/2
    y_center = y1 + h/2

    # Normalize
    x_center *= dw
    w *= dw
    y_center *= dh
    h *= dh

    return [x_center, y_center, w, h]

def convert_ppm_to_jpg(ppm_path, jpg_path):
    """
    Convert PPM image to JPG format.
    """
    try:
        # Read PPM image
        img = cv2.imread(ppm_path)
        if img is None:
            raise Exception(f"Failed to read image: {ppm_path}")

        # Save as JPG
        cv2.imwrite(jpg_path, img, [cv2.IMWRITE_JPEG_QUALITY, 95])
        return True
    except Exception as e:
        print(f"Error converting {ppm_path}: {str(e)}")
        return False

def process_dataset(root_dir, output_dir, train_split=0.8):
    """
    Process the GTSDB dataset and prepare it for YOLO training.

    Args:
        root_dir: Path to the root directory containing TrainCJCNN2013
        output_dir: Path where processed dataset will be saved
        train_split: Percentage of data to use for training
    """
    # Create necessary directories
    create_dirs(output_dir)

    # Read ground truth file
    gt_path = os.path.join(root_dir, 'TrainIJCNN2013', 'gt.txt')
    images_path = os.path.join(root_dir, 'TrainIJCNN2013', '*.ppm')

    # Get all image files
    image_files = glob(images_path)

    # Read annotations
    annotations = {}
    with open(gt_path, 'r') as f:
        for line in f:
            parts = line.strip().split(';')
            img_id = parts[0]
            if img_id not in annotations:
                annotations[img_id] = []

            # Convert coordinates to integers
            x1, y1, x2, y2 = map(int, parts[1:5])
            class_id = int(parts[5])

            annotations[img_id].append((x1, y1, x2, y2, class_id))

    # Split dataset
    image_files = sorted(image_files)
    train_imgs, val_imgs = train_test_split(image_files, train_size=train_split, random_state=42)

    def process_subset(img_paths, subset_type):
        successful_conversions = 0
        for img_path in tqdm(img_paths, desc=f'Processing {subset_type} set'):
            # Get image filename
            img_filename = os.path.basename(img_path)
            jpg_filename = os.path.splitext(img_filename)[0] + '.jpg'

            # Read image to get dimensions
            img = cv2.imread(img_path)
            if img is None:
                print(f"Warning: Could not read image {img_path}")
                continue

            h, w = img.shape[:2]

            # Convert and save image as JPG
            jpg_output_path = os.path.join(output_dir, 'images', subset_type, jpg_filename)
            if convert_ppm_to_jpg(img_path, jpg_output_path):
                successful_conversions += 1
            else:
                continue

            # Create label file
            label_filename = os.path.splitext(img_filename)[0] + '.txt'
            label_path = os.path.join(output_dir, 'labels', subset_type, label_filename)

            # Write annotations in YOLO format
            if img_filename in annotations:
                with open(label_path, 'w') as f:
                    for bbox in annotations[img_filename]:
                        x1, y1, x2, y2, class_id = bbox
                        # Convert to YOLO format
                        yolo_bbox = convert_to_yolo_format((w, h), (x1, y1, x2, y2))
                        # Write to file: class_id center_x center_y width height
                        f.write(f"{class_id} {' '.join([str(x) for x in yolo_bbox])}\n")
            else:
                # Create empty label file if no annotations
                open(label_path, 'w').close()

        return successful_conversions

    # Process train and validation sets
    n_train = process_subset(train_imgs, 'train')
    n_val = process_subset(val_imgs, 'val')

    print(f"\nSuccessfully converted {n_train} training images and {n_val} validation images to JPG format")

    # Create dataset.yaml file
    yaml_content = f"""
path: {output_dir}  # dataset root dir
train: images/train  # train images (relative to 'path')
val: images/val  # val images (relative to 'path')

# Classes
nc: 43  # number of classes
names: ['speed limit 20', 'speed limit 30', 'speed limit 50', 'speed limit 60', 'speed limit 70',
        'speed limit 80', 'restriction ends 80', 'speed limit 100', 'speed limit 120',
        'no passing', 'no passing for vehicles over 3.5 metric tons',
        'right-of-way at the next intersection', 'priority road', 'yield', 'stop',
        'no vehicles', 'vehicles over 3.5 metric tons prohibited', 'no entry',
        'general caution', 'dangerous curve to the left', 'dangerous curve to the right',
        'double curve', 'bumpy road', 'slippery road', 'road narrows on the right',
        'road work', 'traffic signals', 'pedestrians', 'children crossing',
        'bicycles crossing', 'beware of ice/snow', 'wild animals crossing',
        'end of all speed and passing limits', 'turn right ahead', 'turn left ahead',
        'ahead only', 'go straight or right', 'go straight or left', 'keep right',
        'keep left', 'roundabout mandatory', 'end of no passing',
        'end of no passing by vehicles over 3.5 metric tons']
    """

    with open(os.path.join(output_dir, 'dataset.yaml'), 'w') as f:
        f.write(yaml_content.strip())

    return n_train, n_val
def train_yolo(data_yaml_path, epochs=100, batch_size=16, img_size=640):
    """
    Train YOLOv8 model on the traffic sign dataset.

    Args:
        data_yaml_path: Path to dataset.yaml file
        epochs: Number of training epochs
        batch_size: Batch size for training
        img_size: Input image size
    """
    # Initialize YOLOv8 model
    model = YOLO('yolov8n.pt')  # Load pretrained YOLOv8n model

    # Train the model
    results = model.train(
        data=data_yaml_path,
        epochs=epochs,
        batch=batch_size,
        imgsz=img_size,
        patience=20,  # Early stopping patience
        save=True,  # Save best model
        device=0 if torch.cuda.is_available() else 'cpu'
    )

    return results

def main():
    # Set paths
    root_dir = '/content/drive/MyDrive/DLP_A3/TrainIJCNN2013'  # Adjust this to your Google Drive mount point
    output_dir = '/content/traffic_signs_dataset'

    # Process dataset
    n_train, n_val = process_dataset(root_dir, output_dir)
    print(f"Dataset processed: {n_train} training images, {n_val} validation images")

    # Train YOLO model
    data_yaml_path = Path(output_dir) / 'dataset.yaml'
    results = train_yolo(str(data_yaml_path))
    print("Training completed!")

if __name__ == "__main__":
    main()

Processing val set: 100%|██████████| 120/120 [00:05<00:00, 22.86it/s]


Successfully converted 480 training images and 120 validation images to JPG format
Dataset processed: 480 training images, 120 validation images



100%|██████████| 6.25M/6.25M [00:00<00:00, 194MB/s]


Ultralytics 8.3.32 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/traffic_signs_dataset/dataset.yaml, epochs=100, time=None, patience=20, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=Tr

100%|██████████| 755k/755k [00:00<00:00, 42.2MB/s]


...

     52/100      2.22G     0.7624      1.512     0.8101         30        640: 100%|██████████| 30/30 [00:11<00:00,  2.59it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.43it/s]

                   all        120        151      0.403      0.356      0.343      0.262


EarlyStopping: Training stopped early as no improvement observed in last 20 epochs. Best results observed at epoch 32, best model saved as best.pt.
To update EarlyStopping(patience=20) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

52 epochs completed in 0.221 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.32 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,014,033 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.71it/s]


                   all        120        151      0.547      0.326      0.358      0.273
        speed limit 30         10         10      0.552        0.7      0.661      0.573
        speed limit 50         11         12      0.591        0.5      0.529      0.383
        speed limit 60          4          5      0.536        0.4      0.374      0.289
        speed limit 70          2          2          0          0     0.0224     0.0224
        speed limit 80          5          6       0.42       0.14      0.431       0.32
       speed limit 100          2          2      0.187          1      0.638       0.51
       speed limit 120          6          9      0.366      0.444      0.426      0.284
            no passing          5          6          0          0      0.169       0.11
no passing for vehicles over 3.5 metric tons          6         11      0.668          1      0.936      0.731
right-of-way at the next intersection          6          6      0.438      0.333      0

In [17]:

import os
from ultralytics import YOLO
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

class TrafficSignDetector:
    def __init__(self, model_path, conf_threshold=0.25):
        """
        Initialize the traffic sign detector.
        """
        self.model = YOLO(model_path)
        self.conf_threshold = conf_threshold

        # Class names for visualization
        self.class_names = ['speed limit 20', 'speed limit 30', 'speed limit 50', 'speed limit 60',
                           'speed limit 70', 'speed limit 80', 'restriction ends 80', 'speed limit 100',
                           'speed limit 120', 'no passing', 'no passing for vehicles over 3.5 metric tons',
                           'right-of-way at the next intersection', 'priority road', 'yield', 'stop',
                           'no vehicles', 'vehicles over 3.5 metric tons prohibited', 'no entry',
                           'general caution', 'dangerous curve to the left', 'dangerous curve to the right',
                           'double curve', 'bumpy road', 'slippery road', 'road narrows on the right',
                           'road work', 'traffic signals', 'pedestrians', 'children crossing',
                           'bicycles crossing', 'beware of ice/snow', 'wild animals crossing',
                           'end of all speed and passing limits', 'turn right ahead', 'turn left ahead',
                           'ahead only', 'go straight or right', 'go straight or left', 'keep right',
                           'keep left', 'roundabout mandatory', 'end of no passing',
                           'end of no passing by vehicles over 3.5 metric tons']

    def predict_and_visualize(self, image_path, save_path=None):
        """
        Perform detection and visualize results on a single image.
        """
        # Read image
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Could not read image at {image_path}")

        # Convert BGR to RGB
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Perform prediction
        results = self.model.predict(image_rgb, conf=self.conf_threshold)[0]

        # Visualization
        fig, ax = plt.subplots(1, figsize=(12, 8))
        ax.imshow(image_rgb)

        # Plot each detection
        for box in results.boxes:
            # Get box coordinates
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            conf = float(box.conf[0].cpu().numpy())
            cls_id = int(box.cls[0].cpu().numpy())

            # Create rectangle
            rect = plt.Rectangle((x1, y1), x2-x1, y2-y1,
                               fill=False, color='red', linewidth=2)
            ax.add_patch(rect)

            # Add label
            label = f'{self.class_names[cls_id]} ({conf:.2f})'
            ax.text(x1, y1-5, label, color='red',
                   bbox=dict(facecolor='white', alpha=0.8))

        plt.axis('off')

        if save_path:
            plt.savefig(save_path, bbox_inches='tight', pad_inches=0)
            plt.close()
        else:
            plt.show()

        return results

    def evaluate_test_set(self, test_dir, output_dir): #        Evaluate the model on a test set and save visualizations.

        os.makedirs(output_dir, exist_ok=True)

        # Get all test images
        test_images = [f for f in os.listdir(test_dir) if f.endswith('.jpg')]

        results_summary = {
            'total_images': len(test_images),
            'total_detections': 0,
            'detection_confidences': [],
            'detections_per_class': {i: 0 for i in range(len(self.class_names))}
        }

        for img_name in test_images:
            img_path = os.path.join(test_dir, img_name)
            save_path = os.path.join(output_dir, f'detected_{img_name}')

            # Perform detection and visualization
            results = self.predict_and_visualize(img_path, save_path)

            # Collect statistics
            num_detections = len(results.boxes)
            results_summary['total_detections'] += num_detections

            for box in results.boxes:
                cls_id = int(box.cls[0].cpu().numpy())
                conf = float(box.conf[0].cpu().numpy())
                results_summary['detection_confidences'].append(conf)
                results_summary['detections_per_class'][cls_id] += 1

        # Calculate and print summary statistics
        self._print_evaluation_summary(results_summary)

        return results_summary

    def _print_evaluation_summary(self, summary):
        """Print evaluation summary statistics."""
        print("\nEvaluation Summary:")
        print(f"Total images processed: {summary['total_images']}")
        print(f"Total detections: {summary['total_detections']}")

        if summary['detection_confidences']:
            avg_conf = np.mean(summary['detection_confidences'])
            print(f"Average confidence: {avg_conf:.3f}")

        print("\nDetections per class:")
        for cls_id, count in summary['detections_per_class'].items():
            if count > 0:
                print(f"{self.class_names[cls_id]}: {count}")

def main():
    # Paths
    model_path = "runs/detect/train/weights/best.pt"  # Path to your trained model
    test_dir = "/content/traffic_signs_dataset/images/val"  # Your validation/test images
    output_dir = "/content/detection_results"

    # Initialize detector
    detector = TrafficSignDetector(model_path)

    # Run evaluation
    results = detector.evaluate_test_set(test_dir, output_dir)

    print(f"\nResults saved to {output_dir}")

if __name__ == "__main__":
    main()



0: 384x640 (no detections), 37.3ms
Speed: 2.3ms preprocess, 37.3ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 7.4ms
Speed: 5.4ms preprocess, 7.4ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 7.5ms
Speed: 4.0ms preprocess, 7.5ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 restriction ends 80, 1 speed limit 100, 7.5ms
Speed: 2.9ms preprocess, 7.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 7.3ms
Speed: 4.6ms preprocess, 7.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 right-of-way at the next intersection, 7.5ms
Speed: 4.2ms preprocess, 7.5ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 8.2ms
Speed: 3.2ms preprocess, 8.2ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 priority road, 7.2

# Faster RCNN **model**

In [18]:
!pip install torch torchvision albumentations opencv-python numpy tqdm

In [19]:
import os
import cv2
import numpy as np
from glob import glob
import torch
import torchvision
from torchvision import transforms
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import train_test_split
from tqdm import tqdm


In [30]:

class TrafficSignDataset(Dataset):

    def __init__(self, image_paths, annotations, transform=None, train=True):
        self.image_paths = image_paths
        self.annotations = annotations
        self.transform = transform
        self.train = train

    def __getitem__(self, idx):
    # Load image
      img_path = self.image_paths[idx]
      img = cv2.imread(img_path)
      img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

      # Get annotations for this image
      img_id = os.path.basename(img_path)
      boxes = []
      labels = []

      if img_id in self.annotations:
          for x1, y1, x2, y2, class_id in self.annotations[img_id]:
              boxes.append([x1, y1, x2, y2])
              labels.append(class_id)

      # Convert to tensors
      boxes = torch.as_tensor(boxes, dtype=torch.float32)
      labels = torch.as_tensor(labels, dtype=torch.int64)

      target = {
          'boxes': boxes,
          'labels': labels,
          'image_id': torch.tensor([idx]),
          'iscrowd': torch.zeros((len(boxes),), dtype=torch.int64)
      }

      # Calculate areas only if boxes are not empty
      if boxes.shape[0] > 0:
          target['area'] = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
      else:
          target['area'] = torch.tensor([], dtype=torch.float32)

      # Apply transforms
      if self.transform:
          transformed = self.transform(image=img, bboxes=boxes.tolist(), labels=labels.tolist())
          img = transformed['image']
          if len(transformed['bboxes']) > 0:
              target['boxes'] = torch.tensor(transformed['bboxes'], dtype=torch.float32)
              target['labels'] = torch.tensor(transformed['labels'], dtype=torch.int64)
          else:
              target['boxes'] = torch.zeros((0, 4), dtype=torch.float32)
              target['labels'] = torch.zeros(0, dtype=torch.int64)

      return img, target

    def __len__(self):
        return len(self.image_paths)

def get_transform(train=True):
    if train:
        transform = A.Compose([
            A.RandomBrightnessContrast(p=0.5),
            A.RandomGamma(p=0.5),
            A.GaussNoise(p=0.5),
            A.HorizontalFlip(p=0.5),
            A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=15, p=0.5),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))
    else:
        transform = A.Compose([
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))
    return transform

def create_model(num_classes):
    # Load pre-trained model
    model = fasterrcnn_resnet50_fpn(pretrained=True)

    # Replace the classifier with a new one for our number of classes
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model

def process_dataset(root_dir, train_split=0.8):
    """Process the GTSDB dataset for Faster R-CNN training."""
    # Read ground truth file
    gt_path = os.path.join(root_dir, 'TrainIJCNN2013', 'gt.txt')
    images_path = os.path.join(root_dir, 'TrainIJCNN2013', '*.ppm')

    # Get all image files
    image_files = glob(images_path)

    # Read annotations
    annotations = {}
    with open(gt_path, 'r') as f:
        for line in f:
            parts = line.strip().split(';')
            img_id = parts[0]
            if img_id not in annotations:
                annotations[img_id] = []

            # Convert coordinates to integers
            x1, y1, x2, y2 = map(int, parts[1:5])
            class_id = int(parts[5]) + 1  # Add 1 because 0 is background in Faster R-CNN

            annotations[img_id].append((x1, y1, x2, y2, class_id))

    # Split dataset
    train_imgs, val_imgs = train_test_split(image_files, train_size=train_split, random_state=42)

    return train_imgs, val_imgs, annotations

def train_model(model, train_loader, val_loader, num_epochs=10, device='cuda'):
    # Move model to device
    model.to(device)

    # Initialize optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

    # Learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

    for epoch in range(num_epochs):
        # Training
        model.train()
        total_loss = 0

        for images, targets in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            # Compute losses
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())

            optimizer.zero_grad()
            losses.backward()
            optimizer.step()

            total_loss += losses.item()

        # Update learning rate
        lr_scheduler.step()

        # Validation (Skip loss calculation, just evaluate predictions)
        model.eval()
        with torch.no_grad():
            for images, targets in val_loader:
                images = list(image.to(device) for image in images)
                targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

                # Forward pass to check predictions (do not compute loss)
                predictions = model(images)  # Predictions are directly returned in eval mode

        print(f'Epoch {epoch+1}: Train Loss = {total_loss/len(train_loader):.4f}')

    return model

def main():
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Dataset parameters
    root_dir = '/content/drive/MyDrive/DLP_A3/TrainIJCNN2013'
    batch_size = 4
    num_classes = 44  # 43 traffic signs + 1 background class

    # Process dataset
    train_imgs, val_imgs, annotations = process_dataset(root_dir)

    # Create datasets
    train_dataset = TrafficSignDataset(
        train_imgs,
        annotations,
        transform=get_transform(train=True)
    )

    val_dataset = TrafficSignDataset(
        val_imgs,
        annotations,
        transform=get_transform(train=False)
    )

    # Create data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=lambda x: tuple(zip(*x))
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=lambda x: tuple(zip(*x))
    )

    # Create model
    model = create_model(num_classes)

    # Train model
    model = train_model(model, train_loader, val_loader)

    # Save model
    torch.save(model.state_dict(), 'faster_rcnn_traffic_signs.pth')

if __name__ == "__main__":
    main()

Epoch 1/10: 100%|██████████| 120/120 [03:15<00:00,  1.63s/it]


Epoch 1: Train Loss = 0.5204


Epoch 2/10: 100%|██████████| 120/120 [03:14<00:00,  1.62s/it]


Epoch 2: Train Loss = 0.3660


Epoch 3/10: 100%|██████████| 120/120 [03:15<00:00,  1.63s/it]


...

Epoch 8: Train Loss = 0.2560


Epoch 9/10: 100%|██████████| 120/120 [03:17<00:00,  1.65s/it]


Epoch 9: Train Loss = 0.2643


Epoch 10/10: 100%|██████████| 120/120 [03:21<00:00,  1.68s/it]


Epoch 10: Train Loss = 0.2623


# **StreamLit App**

In [ ]:
!streamlit run app.py &


Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: app.py


In [ ]:
# Install dependencies


# Save the Streamlit app as app.py
code = """
import streamlit as st
import cv2
import os
from ultralytics import YOLO
from PIL import Image
import numpy as np

def convert_ppm_to_jpg_streamlit(ppm_file, jpg_output_path):
    try:
        file_bytes = np.asarray(bytearray(ppm_file.read()), dtype=np.uint8)
        img = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)
        if img is None:
            raise Exception("Failed to read image.")
        cv2.imwrite(jpg_output_path, img, [cv2.IMWRITE_JPEG_QUALITY, 95])
        return jpg_output_path
    except Exception as e:
        st.error(f"Error converting image: {str(e)}")
        return None

def predict_yolo(model, image_path):
    try:
        results = model.predict(source=image_path, save=False)
        annotated_img = results[0].plot()
        return annotated_img
    except Exception as e:
        st.error(f"Error during YOLO prediction: {str(e)}")
        return None

st.title("YOLO Traffic Sign Detection")
st.markdown("Upload a PPM image to convert it to JPG and run YOLO predictions.")

uploaded_file = st.file_uploader("Choose a PPM image", type=["ppm"])
if uploaded_file is not None:
    st.image(uploaded_file, caption="Uploaded PPM Image", use_column_width=True)
    st.markdown("### Step 1: Converting to JPG")
    jpg_output_path = "uploaded_image.jpg"
    converted_path = convert_ppm_to_jpg_streamlit(uploaded_file, jpg_output_path)
    if converted_path:
        st.success("Image successfully converted to JPG!")
        st.image(jpg_output_path, caption="Converted JPG Image", use_column_width=True)
        st.markdown("### Step 2: YOLO Prediction")
        with st.spinner("Loading YOLO model..."):
            model = YOLO('/content/runs/detect/train/weights/best.pt')
        st.markdown("### Predictions:")
        annotated_img = predict_yolo(model, jpg_output_path)
        if annotated_img is not None:
            st.image(annotated_img, caption="YOLO Predictions", use_column_width=True)
    else:
        st.error("Image conversion failed. Please try again.")

st.markdown("---")
st.info("Ensure your YOLO model is trained and the weights file (`best.pt`) is available.")
"""
with open("app.py", "w") as file:
    file.write(code)

# Run the Streamlit app
!streamlit run app.py &

# Expose the Streamlit app using ngrok
from pyngrok import ngrok
public_url = ngrok.connect(port=8501)
print(f"Access your app at: {public_url}")





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.2:8501
  External URL: http://34.124.198.47:8501

  Stopping...


Exception in thread Thread-71 (_monitor_process):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pyngrok/process.py", line 139, in _monitor_process
    self._log_line(self.proc.stdout.readline())
  File "/usr/lib/python3.10/encodings/ascii.py", line 26, in decode
    return codecs.ascii_decode(input, self.errors)[0]
UnicodeDecodeError: 'ascii' codec can't decode byte 0xc2 in position 184: ordinal not in range(128)


PyngrokNgrokHTTPError: ngrok client exception, API returned 400: {"error_code":102,"status_code":400,"msg":"invalid tunnel configuration","details":{"err":"yaml: unmarshal errors:\n  line 1: field port not found in type config.HTTPv2Tunnel"}}


In [ ]:
from pyngrok import ngrok
import os

# Set up the streamlit tunnel on port 8501 (default port)
public_url = ngrok.connect(8501)

# Run the Streamlit app in the background
os.system("streamlit run app.py &")

print(f"Streamlit app is live at {public_url}")

Exception in thread Thread-70 (_monitor_process):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pyngrok/process.py", line 139, in _monitor_process
    self._log_line(self.proc.stdout.readline())
  File "/usr/lib/python3.10/encodings/ascii.py", line 26, in decode
    return codecs.ascii_decode(input, self.errors)[0]
UnicodeDecodeError: 'ascii' codec can't decode byte 0xc2 in position 183: ordinal not in range(128)


Streamlit app is live at NgrokTunnel: "https://93bb-34-124-198-47.ngrok-free.app" -> "http://localhost:8501"
